### Sustainability Aware Asset Management: **Groupe A: North America // Scope 1 + 2**
#### **Part I - Standard Asset Allocation**

In [1]:
## Packages lists:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy.optimize import minimize

## Importing all the dataframes from data.ipynb
%run data.ipynb

In [2]:
## Converting into returns using the simple return definition. Also replacing the NaN in the first period by 0.
df_m = monthly_return.pct_change().fillna(0)
df_m.index = pd.to_datetime(df_m.index)

df_y = yearly_return.pct_change().fillna(0)

### Part I - Standard Asset Allocation

#### 1.1 - Construction of a minimum variance portfolio

As we are computing the minimum variance portfolio out-of-sample, we use the first 6 years of monthly returns (from Jan. 2000 to Dec. 2005) to compute the vector of expected returns and the covariance matrix

In [3]:
## Sampling the dataframe from Jan. 2000 to Dec. 2005
start = dt.datetime(2000, 1, 1)
end = dt.datetime(2005, 12, 31)
stocks = df_m.columns
sample_m = df_m[df_m.index.isin(pd.date_range(start, end))]

## Checking if tau is equal to 72
tau = len(sample_m)
tau == 72

True

We computed the expected returns as:
$$
\hat{\mu}_{Y+1} = \frac{1}{\tau}\sum_{k=0}^{\tau-1} R_{t-k}
$$

In [4]:
## It is simply a basic mean function.
mu_hat = pd.DataFrame(sample_m.mean(axis=0)).T

The covariance matrix is computed as:
$$
\Sigma_{Y+1} = \frac{1}{\tau}\sum_{k=0}^{\tau-1} (R_{t-k} - \hat{\mu}_{Y+1})'(R_{t-k} - \hat{\mu}_{Y+1})
$$

In [7]:
## We use the parameter bias=True, because we divide the sum by 1/tau instead of 1/(tau-1)
excess_returns = sample_m.subtract(mu_hat.values.squeeze(), axis=1)
covmat = 1/tau * excess_returns.T @ excess_returns

## We control if there is a missing value in our covariance matrix
print(covmat.isnull().values.any())

False


For the allocation, we use the following maximizazion problem, while restricting the optimal weights to be positive:

$$\min_{{{\alpha_{Y}}}}\quad \sigma^{2}_{p,Y+1} = \alpha'_{Y}\Sigma_{Y+1}\alpha_{Y}$$

$$\text{s.t.}\quad \alpha'_{Y}e = 1$$

$$\text{s.t.}\quad  \alpha_{i,Y} \ge 0 \quad \text{for all i}$$

In [16]:
# Define objective function (portfolio variance)
def portfolio_variance(weights, covmat):
    return np.dot(weights.T, np.dot(covmat, weights))

# Define constraint (sum of weights equals 1)
def constraint(weights):
    return np.sum(weights) - 1
    
# Number of assets
n_assets = len(covmat)

# Initial guess for weights
initial_weights = np.ones(n_assets) / n_assets

# Define bounds for weights (0 to 1) long-only portfolio
bounds = [(0, None)] * n_assets

# Perform optimization
result = minimize(portfolio_variance, initial_weights, args=(covmat,), constraints={'type': 'eq', 'fun': constraint}, bounds=bounds)

# Get optimal weights
optimal_weights = result.x

# Print results
weights = pd.DataFrame(optimal_weights, index=covmat.index, columns=['weight'])
weights

,weight
SCHLUMBERGER,0.000000e+00
ARCH CAP.GP.,0.000000e+00
EVEREST GROUP,3.151424e-18
NABORS INDUSTRIES,0.000000e+00
RENAISSANCERE HDG.,1.160498e-02
...,...
XCEL ENERGY,0.000000e+00
XEROX HOLDINGS,0.000000e+00
YUM! BRANDS,3.882757e-04
ZEBRA TECHNOLOGIES 'A',2.882713e-03


In [ ]:
## Rebalance portfolio from Dec. 2005 to Dec. 2021

In [17]:
def roll_min_var_opt(df, start, end):
    stocks = df.columns
    year_weight = end.year + 1
    weights_df = pd.DataFrame(index=stocks)

    for i in range(17):
        ## Resample from start to end
        sample_m = df[df.index.isin(pd.date_range(start, end))]
        tau = len(sample_m)

        ## Compute expected returns
        mu_hat = pd.DataFrame(sample_m.mean(axis=0)).T
    
        ## Excess returns
        excess_returns = sample_m.subtract(mu_hat.values.squeeze(), axis=1)
    
        ## Covariance Matrix
        covmat = 1/tau * excess_returns.T @ excess_returns
    
        # Define objective function (portfolio variance)
        def portfolio_variance(weights, covmat):
            return np.dot(weights.T, np.dot(covmat, weights))
    
        # Define constraint (sum of weights equals 1)
        def constraint(weights):
            return np.sum(weights) - 1
    
        n_assets = len(covmat)
    
        # Initial guess for weights
        initial_weights = np.ones(n_assets) / n_assets
        
        # Define bounds for weights (0 to 1) long-only portfolio
        bounds = [(0, None)] * n_assets
    
        result = minimize(portfolio_variance, initial_weights, args=(covmat,), constraints={'type': 'eq', 'fun': constraint}, bounds=bounds)
    
        optimal_weights = result.x

        weights_df[year_weight] = optimal_weights

        start = dt.datetime(start.year + 1, 1, 1)
        end = dt.datetime(end.year + 1, 1, 1)
        year_weight += 1

    return weights_df

In [20]:
start = dt.datetime(2000, 1, 1)
end = dt.datetime(2005, 12, 31)

weights_df = roll_min_var_opt(df_m, start, end)

In [22]:
weights_df.head()

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
SCHLUMBERGER,0.000000e+00,2.553878e-18,2.023303e-18,2.805777e-19,0.000000e+00,1.208465e-18,0.000000,1.409217e-17,0.000000e+00,1.985693e-18,1.755540e-18,3.277177e-18,0.000000e+00,0.000000,0.000000,3.369771e-18,0.000000e+00
ARCH CAP.GP.,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.010143,0.000000e+00,0.000000e+00,2.780385e-18,0.000000e+00,0.000000e+00,3.182660e-18,0.001013,0.001145,0.000000e+00,0.000000e+00
EVEREST GROUP,3.151424e-18,0.000000e+00,0.000000e+00,5.056417e-19,4.109678e-19,7.963530e-18,0.000208,0.000000e+00,4.909969e-18,3.121960e-03,0.000000e+00,0.000000e+00,9.010329e-03,0.002662,0.044626,6.702413e-02,2.534687e-02
NABORS INDUSTRIES,0.000000e+00,2.549240e-18,0.000000e+00,1.457126e-19,3.816951e-19,3.885951e-18,0.000000,6.884807e-18,2.371563e-18,0.000000e+00,3.227538e-18,1.313765e-19,0.000000e+00,0.000000,0.000000,0.000000e+00,1.546220e-18
RENAISSANCERE HDG.,1.160498e-02,1.824573e-02,2.294535e-02,2.284458e-18,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,3.260270e-02,1.938555e-02,3.693503e-03,9.341518e-03,0.003801,0.012310,3.082429e-02,3.846325e-03


In [27]:
## Checking if the first constraint is respected.
weights_df.sum()

2006    1.0
2007    1.0
2008    1.0
2009    1.0
2010    1.0
2011    1.0
2012    1.0
2013    1.0
2014    1.0
2015    1.0
2016    1.0
2017    1.0
2018    1.0
2019    1.0
2020    1.0
2021    1.0
2022    1.0
dtype: float64

In [36]:
## Checking if the second constraint is respected.
print('The smallest weight is: ', np.min(weights_df))
print('The highest weight is: ', np.max(weights_df))

The smallest weight is:  0.0
The highest weight is:  0.15300345822319314


In [ ]:
## Compute the ex-post performance of the portfolio